In [8]:
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb

folder = 'C:/Users/danne/Documents/UofT/FamousRecon'
folder = 'D:/Ilya_study/Analysis'
import numpy as np
import pandas as pd
perc_ind=['02','03','04','05','06','07','08','09','10','11','12']
imag_ind=['02','03','04','05','06','07','08','09','10','11','12']
perc_file=[folder+'/conf_mat_perc_'+ i +'.csv' for i in perc_ind]
imag_file=[folder+'/conf_IR_'+ i +'_imag.csv' for i in perc_ind]
data = load_multiple_conf_mats(folder, perc_file, imag_file)
perc_list = data[0]
imag_list = data[1]

In [9]:
new_perc = [df.values for df in perc_list]
a = np.array(new_perc)
print(a.shape)
a = np.mean(a, axis=0)
print(a.shape)
a = pd.DataFrame(a)
perc_list=[]
perc_list.append(a)

(11, 100, 100)
(100, 100)


In [10]:
new_imag = [df.values for df in imag_list]
a = np.array(new_imag)
print(a.shape)
a = np.mean(a, axis=0)
print(a.shape)
a = pd.DataFrame(a,index=['mcy','sgo','sjo','est','tsw'],columns=['mcy','sgo','sjo','est','tsw'])
imag_list=[]
imag_list.append(a)

(11, 5, 5)
(5, 5)


In [11]:
imag_list[0]

,mcy,sgo,sjo,est,tsw
mcy,0.000000,0.573864,0.647727,0.587121,0.585227
sgo,0.573864,0.000000,0.560606,0.613636,0.632576
sjo,0.647727,0.560606,0.000000,0.621212,0.710227
est,0.587121,0.613636,0.621212,0.000000,0.659091
tsw,0.585227,0.632576,0.710227,0.659091,0.000000


In [14]:
import pdb;
import os
os.chdir('C:\\Users\\danne\\Dropbox\\Programming\\Python\\Work\\EEG_analysis')
mat = inter_sub_corr(perc_list[0], imag_list[0])

Correlation between upright percieved and imagary discrimination is 0.03
Correlation between inverted percieved and imagary discriminations is 0.00
Correlation between imagery and TO 0.11
Correlation between upright percieved and TO discriminations is 0.23
Correlation between inverted percieved and TO discriminations is 0.23
Correlation between unfamiliar upright percieved and TO discriminations is 0.27
Correlation between famous upright percieved and TO discriminations is 0.21
Correlation between unfamiliar inverted percieved and TO discriminations is 0.19
Correlation between famous inverted percieved and TO discriminations is 0.27


In [ ]:
os.chdir('C:\\Users\\danne\\Dropbox\\Programming\\Python\\Work\\EEG_analysis')

In [13]:
def inter_sub_corr(df_perc, df_imag):
    from scipy.spatial.distance import pdist, squareform
    %run general_tools.ipynb
    
    im_names = ['mcy','sgo','sjo','est','tsw']
    corr_names = ['est','mcy','sjo','sgo','tsw']
    pr_names = ['adi','ani','ama','ago','aza','ekl','evu','epo','eiv','ech','ian','jpi','kda',
               'kgo','mbo','mbe','ofa','pan','pga','rga','siv','tar','tka','yst','ype','ase',
               'aha','ake','cmo','eol','epa','ecl','ero','est','ewa','jla','jal','kpe','kkn',
                'kst','mcy','ndo','npo','owi','pcr','rmc','rwi','sjo','sgo','tsw']
    
    # Theoretical observer:
    folder1 = 'C:/Users/danne/Documents/UofT/FamousRecon/set3/east'
    folder2 = 'C:/Users/danne/Documents/UofT/FamousRecon/set3/west'
    df = pd.concat((image_to_df(folder1),image_to_df(folder2)))
    df['origin'] = ['east']*25+['west']*25
    conf = squareform(pdist(df.iloc[:,:16428],'euclidean'))
    df_to = pd.DataFrame(conf, columns = pr_names, index = pr_names)
    #pdb.set_trace()
    df_perc.columns = [pr_names*2]
    df_perc.index = [pr_names*2]
    idx = pd.IndexSlice
    df_perc_imag = df_perc.loc[idx[im_names], idx[im_names]]
    df_imag = df_imag.reindex(corr_names)
    df_imag = df_imag.transpose().reindex(corr_names).transpose()
    df_to_imag = df_to.loc[idx[im_names], idx[im_names]]
    df_to_imag = df_to_imag.reindex(corr_names)
    df_to_imag = df_to_imag.transpose().reindex(corr_names).transpose()
    #pdb.set_trace()
    # compute correlations
    up_corr = np.corrcoef(squareform(df_imag),squareform(df_perc_imag.iloc[:5,:5].values))
    print(f'Correlation between upright percieved and imagary discrimination is {up_corr[1,0]:.2f}')
    in_corr = np.corrcoef(squareform(df_imag),squareform(df_perc_imag.iloc[-5:,-5:].values))
    print(f'Correlation between inverted percieved and imagary discriminations is {in_corr[1,0]:.2f}')

    to_imag_corr = np.corrcoef(squareform(df_to_imag),squareform(df_imag.iloc[:5,:5].values))
    print(f'Correlation between imagery and TO {to_imag_corr[1,0]:.2f}')


    up_perc_to_corr = np.corrcoef(squareform(df_to.values),squareform(df_perc.iloc[:50,:50].values))
    print(f'Correlation between upright percieved and TO discriminations is {up_perc_to_corr[1,0]:.2f}')
    in_perc_to_corr = np.corrcoef(squareform(df_to.values),squareform(df_perc.iloc[50:100,50:100].values))
    print(f'Correlation between inverted percieved and TO discriminations is {in_perc_to_corr[1,0]:.2f}')
    up_perc_to_corr_unf = np.corrcoef(squareform(df_to.values[:25,:25]),squareform(df_perc.iloc[:25,:25].values))
    print(f'Correlation between unfamiliar upright percieved and TO discriminations is {up_perc_to_corr_unf[1,0]:.2f}')
    up_perc_to_corr_fam = np.corrcoef(squareform(df_to.values[25:50,25:50]),squareform(df_perc.iloc[25:50,25:50].values))
    print(f'Correlation between famous upright percieved and TO discriminations is {up_perc_to_corr_fam[1,0]:.2f}')

    in_perc_to_corr_unf = np.corrcoef(squareform(df_to.values[:25,:25]),squareform(df_perc.iloc[50:75,50:75].values))
    print(f'Correlation between unfamiliar inverted percieved and TO discriminations is {in_perc_to_corr_unf[1,0]:.2f}')
    in_perc_to_corr_fam = np.corrcoef(squareform(df_to.values[25:50,25:50]),squareform(df_perc.iloc[75:100,75:100].values))
    print(f'Correlation between famous inverted percieved and TO discriminations is {in_perc_to_corr_fam[1,0]:.2f}')
    return df_perc_imag

def load_multiple_conf_mats(folder, perc_file, imag_file):
    perc_list=[]
    imag_list=[]
    for i,j in zip(perc_file,imag_file):
        perc_list.append(pd.read_csv(i,header=None))
        imag_list.append(pd.read_csv(j, index_col = 0))
    return(perc_list,imag_list)

def image_to_df(folder, extension = 'tif'):
    '''Loads all images in the folder, removes backgrounds and returns a 
         pandas data frame with column 'names' as file names
    Args: 
    :param folder: the folder with images
    :param extension: image extension. Default - tif
    '''
    
    import os
    import glob
    from PIL import Image
    import numpy as np
    import pandas as pd

    cwd = os.getcwd()
    # definig the folder
    os.chdir(folder)
    # loading images
    image_list = []
    flList=list(glob.glob('*.'+extension))
    flList.sort()
    for filename in flList: 
        im = Image.open(filename)
        image_list.append(np.array(im))
    data=[]
    [data.append(np.array(np.asarray(i)).ravel()) for i in image_list]
    data=np.array(data)
    mask=np.sum(data,0)==0
    data=data[:,np.logical_not(mask)]
    feature_num=data.shape[1]
    df=pd.DataFrame(data)
    df['names']=flList
    df.names = df.names.str[:-6]
    os.chdir(cwd)
    return df

In [ ]:
import numpy as np
from PIL import Image
im = Image.open('C:/Users/danne/Documents/UofT/FamousRecon/set3/east/agni diko 4.tif')
a=np.array(im)
type(a)